In [2]:
import pandas as pd
import plotly.express as px

In [6]:
orders = pd.read_csv('data/orders.csv')
order_items = pd.read_csv('data/order_items.csv')
order_item_refunds = pd.read_csv('data/order_item_refunds.csv')
products = pd.read_csv('data/products.csv')

In [ ]:
orders['created_at'] = pd.to_datetime(orders['created_at'], errors='coerce')
order_items['created_at'] = pd.to_datetime(order_items['created_at'], errors='coerce')
order_item_refunds['created_at'] = pd.to_datetime(order_item_refunds['created_at'], errors='coerce')

In [ ]:

df = (
    order_items[['order_item_id', 'order_id', 'product_id']].merge(orders[['order_id', 'created_at', 'price_usd']],on='order_id', how='left')
    .merge(products[['product_id', 'product_name']],on='product_id', how='left'))

refunds = (order_item_refunds[['order_item_id', 'created_at', 'refund_amount_usd']].merge(order_items[['order_item_id', 'product_id']], on='order_item_id', how='left')
    .merge(products[['product_id', 'product_name']], on='product_id', how='left')
)


In [20]:
df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce')
refunds['created_at'] = pd.to_datetime(refunds['created_at'], errors='coerce')
orders['created_at'] = pd.to_datetime(orders['created_at'], errors='coerce')
order_item_refunds['created_at'] = pd.to_datetime(order_item_refunds['created_at'], errors='coerce')


Total units sold

In [10]:
data = df['product_name'].value_counts().reset_index()
data.columns = ['product_name', 'units_sold']

fig1 = px.bar(data, x='product_name', y='units_sold', title='Total Units Sold per Product')
fig1.show()


* Shows the total quantity sold for each product.
* The Original Mr. Fuzzy has the highest sales volume.
* Other products have much lower but similar sales levels.

In [51]:
product_sales = (
    df.groupby("product_name")["price_usd"].sum().reset_index(name="total_sales_usd").sort_values("total_sales_usd", ascending=False)
)

px.bar(product_sales,x="product_name",y="total_sales_usd",title="Top Products by Total Sales (USD)").show()


* Shows total revenue generated by each product.
* The Original Mr. Fuzzy contributes the highest revenue.
* Remaining products contribute smaller portions of total sales.

Refund % per product (per month: slider) heat map

In [52]:
df['month'] = df['created_at'].dt.to_period('Y').astype(str)
refunds['month'] = refunds['created_at'].dt.to_period('Y').astype(str)

total_pm = df.groupby(['product_name', 'month']).size().reset_index(name='total')
refund_pm = refunds.groupby(['product_name', 'month']).size().reset_index(name='refunds')

rate = total_pm.merge(refund_pm, on=['product_name', 'month'], how='left').fillna(0)
rate['refund_pct'] = (rate['refunds'] / rate['total'] * 100).round(2)


heat = rate.pivot(index='product_name', columns='month', values='refund_pct').fillna(0)
fig2 = px.imshow(
    heat,
    text_auto=True,  
    color_continuous_scale='Blues',
    title='Refund % per Product per Month'
)
fig2.update_layout(
    xaxis_title='Year',
    yaxis_title='Product'
)


fig2.show()


* Shows refund percentage for each product across different years.
* Darker colors indicate higher refund rates.
* Helps identify problematic products and time periods with more refunds.

In [42]:
rate

,product_name,month,total,refunds,refund_pct
0,The Birthday Sugar Panda,2013-12,139,6.0,4.316547
1,The Birthday Sugar Panda,2014-01,200,13.0,6.500000
2,The Birthday Sugar Panda,2014-02,211,14.0,6.635071
3,The Birthday Sugar Panda,2014-03,244,14.0,5.737705
4,The Birthday Sugar Panda,2014-04,267,18.0,6.741573
...,...,...,...,...,...
89,The Original Mr. Fuzzy,2014-11,1451,31.0,2.136458
90,The Original Mr. Fuzzy,2014-12,1584,75.0,4.734848
91,The Original Mr. Fuzzy,2015-01,1392,50.0,3.591954
92,The Original Mr. Fuzzy,2015-02,1113,42.0,3.773585


In [44]:
import plotly.express as px

df['month'] = df['created_at'].dt.to_period('M').astype(str)
refunds['month'] = refunds['created_at'].dt.to_period('M').astype(str)

total_pm = df.groupby(['product_name', 'month']).size().reset_index(name='total')
refund_pm = refunds.groupby(['product_name', 'month']).size().reset_index(name='refunds')
rate = total_pm.merge(refund_pm, on=['product_name', 'month'], how='left').fillna(0)
rate['refund_pct'] = (rate['refunds'] / rate['total'] * 100).round(1)
heat = rate.pivot(index='product_name', columns='month', values='refund_pct').fillna(0)
fig2 = px.imshow(
    heat,
    text_auto=True,  
    color_continuous_scale='Blues',
    title='Refund % per Product per Month'
)
fig2.update_layout(
    xaxis_title='Month',
    yaxis_title='Product'
)

fig2.show()


* Shows refund percentage for each product across different months.
* Darker colors indicate higher refund rates.
* Helps identify problematic products and time periods with more refunds.

Revenue trends, number of orders, refund order values ( line graph)

In [23]:
orders['month'] = orders['created_at'].dt.to_period('M').astype(str)
order_item_refunds['month'] = order_item_refunds['created_at'].dt.to_period('M').astype(str)
revenue_trend = orders.groupby('month')['price_usd'].sum().reset_index(name='revenue')

orders_trend = orders.groupby('month').size().reset_index(name='orders_count')

refund_trend = order_item_refunds.groupby('month')['refund_amount_usd'].sum().reset_index(name='refund_amount')

trend = revenue_trend.merge(orders_trend, on='month', how='left').merge(refund_trend, on='month', how='left').fillna(0)


In [33]:
import plotly.express as px

fig = px.line(trend,x='month',y=['revenue', 'orders_count', 'refund_amount'],title='Revenue, Orders, and Refund Trends Over Time',markers=True)

fig.show()


* Shows trends of revenue, number of orders, and refund amount over time.
* Revenue and orders show an overall increasing trend.
* Refund amounts fluctuate and show some spikes in certain periods.

New Vs Repeat Customer

In [ ]:
cust_orders = orders.groupby('user_id').size()
new_customers = (cust_orders == 1).sum()
repeat_customers = (cust_orders > 1).sum()
data = {'type': ['New Customers', 'Repeat Customers'],'count': [new_customers, repeat_customers]}
fig = px.pie(names=data['type'],values=data['count'],title='New vs Repeat Customers')
fig.show()


* Compares new customers and repeat customers.
* Majority of customers are new customers.
* Indicates good customer acquisition but low repeat purchase rate.

In [32]:
refund_product = refunds.groupby('product_name')['refund_amount_usd'].sum().reset_index()

px.pie(refund_product,names='product_name',values='refund_amount_usd',title='Refund Amount per Product').show()


* Shows how total refund amount is distributed across products.
* The Original Mr. Fuzzy has the highest share of refunds.
* Other products contribute smaller portions to total refund cost.